# Performance comparison studies

This notebook contains the code to reproduce the results of the performance comparison studies for different BDT and DNN-based models presented in the paper (Figure 7) as well as a study on the impact of a random rotation on a BDT and a DNN model (Figure 8). 

# Study 1: Performance comparison of different BDT and DNN-based models

There are two options to run the code for this study:

1. Train all the models yourself. While this is certainly possible with this code, it will require quite some time to train all the ensembles of all the different model types. Consider extracting the code in the respective cells to separate python scripts and run them on a computing cluster.

2. Use the pre-trained models. The pre-trained models are available in the `treebased_ad_files/models.zip` file. The code in this notebook will extract the model files and use them for the performance comparison study.

**Important note:** For extracting the pre-trained models, it is important to use the exact same version of the python packages that were used to train the models. Therefore, create a `conda` environment using the `environment.yml` file and run the notebook from within this environment if you want to load the models.

### Option 1: Train all models yourself

First, we import the necessary functions that we'll need for this notebook and then load the data:

In [ ]:
from utils import (load_lhco_rd, add_gaussian_features, train_model_multirun,
                   load_models_allruns, eval_ensemble, multi_roc_sigeffs,
                   random_rotation)
from plot_utils import plot_sic_curves, plot_sic_curve_comparison
from tmva_utils import train_tmva_multi, eval_tmva_multi
from os.path import exists, join
from os import mkdir
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.cm as col

**Note:** If the below cell gives you loading errors (containing an error message saying that the data cannot be loaded when `allow_pickle=False`), go to the `treebased_ad_files` folder in your terminal and type the following commands:
```bash
git lfs install
git lfs pull
```
Then restart the kernel of this notebook and try to run again.

In [ ]:
# load data
data = load_lhco_rd("./treebased_ad_files/lhco_rd")

### Step 1.1: Train Histogram Gradient Boosting (HGB) models


First, we set the parameters for all the models in terms of how many training runs we want to do and how many models should be contained in a single ensemble. The settings in the cell below reflect those used in the paper.

In [ ]:
# Re-run training with five Gaussian noise features added
# How often to re-run the entire ensemble training procedure
num_runs = 10

# How many models constitute a single ensemble
ensembles_per_model = 10

max_iters = 100

Now we train the HGB models, firstly on the original dataset, containing four physics features:

In [ ]:
# Train HGB models
full_losses_hgb_0G, models_hgb_0G = train_model_multirun(
    data,
    num_runs=num_runs, ensembles_per_model=ensembles_per_model,
    max_iters=max_iters, model_type="HGB", compute_val_weights=True,
    save_model_dir="./models/models_hgb_0G",
    cv_mode="random", early_stopping=True)

Next, we train the HGB models on the dataset containing the four physics features and the ten additional features, which are pure Gaussian noise:

In [ ]:
# Add noise features
data_10G = add_gaussian_features(data, 10)

In [ ]:
# Train HGB models
full_losses_hgb_10G, models_hgb_10G = train_model_multirun(
    data_10G,
    num_runs=num_runs, ensembles_per_model=ensembles_per_model,
    max_iters=max_iters, model_type="HGB", compute_val_weights=True,
    save_model_dir="./models/models_hgb_10G",
    cv_mode="random", early_stopping=True)

### Step 1.2: Train Adaboost models

Again, we first train on the original dataset:

In [ ]:
# Train Adaboost models
full_losses_ada_0G, models_ada_0G = train_model_multirun(
    data,
    num_runs=num_runs, ensembles_per_model=ensembles_per_model,
    max_iters=max_iters, model_type="Ada", compute_val_weights=True,
    save_model_dir="./models/models_ada_0G",
    cv_mode="random", early_stopping=False)

Then we train on the dataset using the ten Gaussian features added:

In [ ]:
# Train Adaboost models
full_losses_ada_10G, models_ada_10G = train_model_multirun(
    data_10G,
    num_runs=num_runs, ensembles_per_model=ensembles_per_model,
    max_iters=max_iters, model_type="Ada", compute_val_weights=True,
    save_model_dir="./models/models_ada_10G",
    cv_mode="random", early_stopping=False)

### Step 1.3: Train random forest models

First on "vanilla" dataset:

In [ ]:
# Train random forest models
full_losses_rf_0G, models_rf_0G = train_model_multirun(
    data,
    num_runs=num_runs, ensembles_per_model=ensembles_per_model,
    max_iters=max_iters, model_type="RF", compute_val_weights=True,
    save_model_dir="./models/models_rf_0G",
    cv_mode="random", early_stopping=False)

Then on dataset with ten Gaussian features added:

In [ ]:
# Train random forest models
full_losses_rf_10G, models_rf_10G = train_model_multirun(
    data_10G,
    num_runs=num_runs, ensembles_per_model=ensembles_per_model,
    max_iters=max_iters, model_type="RF", compute_val_weights=True,
    save_model_dir="./models/models_rf_10G",
    cv_mode="random", early_stopping=False)

### Step 1.4: Train ROOT TMVA models

Again, first on the dataset with the four physics features. Note that the interface for the ROOT models is slightly different than for the other models.

In [ ]:
_ = train_tmva_multi(
    data,
    num_runs=num_runs, ensembles_per_model=ensembles_per_model,
    cv_mode="random",
    model_identifier="BDT", root_file_dir_base="./models/TMVA_0G")

predictions_tmva_0G = eval_tmva_multi(
    data, 
    num_runs=num_runs, ensembles_per_model=ensembles_per_model, model_identifier="BDT",
    root_file_dir_base="./models/TMVA_0G", save_ensemble_preds=True)

And again on the dataset containing the ten Gaussian features:

In [ ]:
_ = train_tmva_multi(
    data_10G,
    num_runs=num_runs, ensembles_per_model=ensembles_per_model,
    cv_mode="random",
    model_identifier="BDT", root_file_dir_base="./models/TMVA_10G")

predictions_tmva_10G = eval_tmva_multi(
    data_10G, 
    num_runs=num_runs, ensembles_per_model=ensembles_per_model, model_identifier="BDT",
    root_file_dir_base="./models/TMVA_10G", save_ensemble_preds=True)

### Step 1.5: Train DNN models

We start with the "vanilla" dataset:

In [ ]:
# Train DNN models
full_losses_dnn_0G, models_dnn_0G = train_model_multirun(
    data,
    num_runs=num_runs, ensembles_per_model=ensembles_per_model,
    max_iters=max_iters, model_type="DNN", compute_val_weights=True,
    save_model_dir="./models/models_dnn_0G",
    cv_mode="random", early_stopping=True)

And again the dataset with 10 Gaussian noise features added:

In [ ]:
# Train DNN models
full_losses_dnn_10G, models_dnn_10G = train_model_multirun(
    data_10G,
    num_runs=num_runs, ensembles_per_model=ensembles_per_model,
    max_iters=max_iters, model_type="DNN", compute_val_weights=True,
    save_model_dir="./models/models_dnn_10G",
    cv_mode="random", early_stopping=True)

### Optional step: Instead of training the models yourself, load the pre-trained models

In [ ]:
# Extract models if not already done
if not exists("treebased_ad_files/models"):
    from utils import extract_models
    extract_models()

### Load model files
**NOTE:** We load all models except for the RF and the TMVA models, since there the model files are just too large to be stored on the repository. However, we stored the signal and background efficiencies for these models instead, so that we can still compare them to the other models.

In [ ]:
models_ada_0G = load_models_allruns("./treebased_ad_files/models/models_ADA_0G")
models_ada_10G = load_models_allruns("./treebased_ad_files/models/models_ADA_10G")
models_hgb_0G = load_models_allruns("./treebased_ad_files/models/models_HGB_0G")
models_hgb_10G = load_models_allruns("./treebased_ad_files/models/models_HGB_10G")

# load signal efficiency and background efficiency (tpr and fpr) for random forest and TMVA models
tpr_rf_0G = np.load("./treebased_ad_files/models/models_RF_0G/tpr_RF_0G.npy")
fpr_rf_0G = np.load("./treebased_ad_files/models/models_RF_0G/fpr_RF_0G.npy")
tpr_rf_10G = np.load("./treebased_ad_files/models/models_RF_10G/tpr_RF_10G.npy")
fpr_rf_10G = np.load("./treebased_ad_files/models/models_RF_10G/fpr_RF_10G.npy")

tpr_tmva_0G = np.load("./treebased_ad_files/models/models_TMVA_0G/tpr_TMVA_0G.npy")
fpr_tmva_0G = np.load("./treebased_ad_files/models/models_TMVA_0G/fpr_TMVA_0G.npy")
tpr_tmva_10G = np.load("./treebased_ad_files/models/models_TMVA_10G/tpr_TMVA_10G.npy")
fpr_tmva_10G = np.load("./treebased_ad_files/models/models_TMVA_10G/fpr_TMVA_10G.npy")


For loaded models, run ensemble evaluation and also comput the signal and background efficiencies:

**NOTE:** The following cell will take a while to run. Last time it was tested, it took around 10 minutes on a modern-era CPU

In [ ]:
# Get model predictions
predictions_ada_0G = eval_ensemble(models_ada_0G, data, model_type="Ada")
predictions_ada_10G = eval_ensemble(models_ada_10G, data_10G, model_type="Ada")
predictions_hgb_0G = eval_ensemble(models_hgb_0G, data, model_type="HGB")
predictions_hgb_10G = eval_ensemble(models_hgb_10G, data_10G, model_type="HGB")

# Compute signal efficiency and background efficiency (tpr and fpr) for all models
tpr_ada_0G, fpr_ada_0G = multi_roc_sigeffs(predictions_ada_0G, data["y_test"])
tpr_ada_10G, fpr_ada_10G = multi_roc_sigeffs(predictions_ada_10G, data_10G["y_test"])
tpr_hgb_0G, fpr_hgb_0G = multi_roc_sigeffs(predictions_hgb_0G, data["y_test"])
tpr_hgb_10G, fpr_hgb_10G = multi_roc_sigeffs(predictions_hgb_10G, data_10G["y_test"])


### Step 1.6: Create SIC curves for all models

We start again on the initial dataset

In [ ]:
# set RC params
plt.rcParams['pgf.rcfonts'] = False
plt.rcParams['font.serif'] = []
plt.rcParams['axes.formatter.useoffset'] = False
plt.rcParams['lines.linewidth'] = 2
plt.rcParams['errorbar.capsize'] = 2
plt.rcParams['grid.linewidth'] = 0.5
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['axes.titlesize'] = 16
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
plt.rcParams['legend.title_fontsize'] = 12
plt.rcParams['legend.fontsize'] = 12
plt.rcParams['legend.frameon'] = False

color_list = ["black", "red", "orange", "dodgerblue"]


In [ ]:
if not exists("plots"):
    mkdir("plots")

plot_sic_curves([tpr_hgb_0G, tpr_rf_0G, tpr_ada_0G, tpr_tmva_0G],
                [fpr_hgb_0G, fpr_rf_0G, fpr_ada_0G, fpr_tmva_0G],
                5*[data["y_test"]],
                out_filename=join("plots", "model_comparison_0G.pdf"),
                labels=["HGB", "RF", "Adaboost", "TMVA BDT"],
                xlabel=r"$\epsilon_{S}$",
                color_list=color_list,
                title="Baseline",
                ylabel=r"$\epsilon_S/\sqrt{\epsilon_B}$",
                max_y=20)

And then on the dataset with the ten Gaussian noise features added:

In [ ]:
plot_sic_curves([tpr_hgb_10G, tpr_rf_10G, tpr_ada_10G, tpr_tmva_10G],
                [fpr_hgb_10G, fpr_rf_10G, fpr_ada_10G, fpr_tmva_10G],
                5*[data_10G["y_test"]],
                out_filename=join("plots", "rotation_comparison_10G.pdf"),
                labels=["HGB", "RF", "Adaboost", "TMVA BDT"],
                xlabel=r"$\epsilon_{S}$",
                color_list=color_list,
                title="Baseline + 10G",
                ylabel=r"$\epsilon_S/\sqrt{\epsilon_B}$",
                max_y=20)

# Study 2: Impact of random rotation on BDT and DNN models

Testing the performance of BDT and DNN models under random rotations of the input data. The cells below can be used to reproduce figure 8 from the paper.

Again there are two ways to run the code:
- either train the models yourself (which will take some time and optimally should be conducted on a computing cluster)
- or load the pre-trained models

First, let's load the data:

In [ ]:
# Load data - for this comparison, we need the original data as well as the data with three Gaussian noise features added
data = load_lhco_rd("./treebased_ad_files/lhco_rd")
data_3G = add_gaussian_features(data, 3)

# We also need the same data but with a random rotation applied to the features
data_rotated = random_rotation(data)
data_3G_rotated = random_rotation(data_3G)

### Option 1: Train all models yourself

### Step 2.1: Train HGB models

Use the following settings for all trainings in this study:

In [ ]:
# Re-run training with five Gaussian noise features added
# How often to re-run the entire ensemble training procedure
num_runs = 10

# How many models constitute a single ensemble
ensembles_per_model = 10

max_iters = 100

We start by training HGB models on the initial dataset:

In [ ]:
# Train HGB models
full_losses_hgb_0G, models_hgb_0G = train_model_multirun(
    data,
    num_runs=num_runs, ensembles_per_model=ensembles_per_model,
    max_iters=max_iters, model_type="HGB", compute_val_weights=True,
    save_model_dir="./models/models_hgb_0G",
    cv_mode="random", early_stopping=True)

And then on the dataset with the three Gaussian noise features added:

In [ ]:
# Train HGB models
full_losses_hgb_3G, models_hgb_3G = train_model_multirun(
    data_3G,
    num_runs=num_runs, ensembles_per_model=ensembles_per_model,
    max_iters=max_iters, model_type="HGB", compute_val_weights=True,
    save_model_dir="./models/models_hgb_3G",
    cv_mode="random", early_stopping=True)

Now we train the same models, but this time on the rotated features. We start again with the initial dataset:

In [ ]:
# Train HGB models
full_losses_hgb_0G_rotated, models_hgb_0G_rotated = train_model_multirun(
    data_rotated,
    num_runs=num_runs, ensembles_per_model=ensembles_per_model,
    max_iters=max_iters, model_type="HGB", compute_val_weights=True,
    save_model_dir="./models/models_hgb_0G_rotated",
    cv_mode="random", early_stopping=True)

And then on the dataset with the three Gaussian noise features added:

In [ ]:
# Train HGB models
full_losses_hgb_3G_rotated, models_hgb_3G_rotated = train_model_multirun(
    data_3G_rotated,
    num_runs=num_runs, ensembles_per_model=ensembles_per_model,
    max_iters=max_iters, model_type="HGB", compute_val_weights=True,
    save_model_dir="./models/models_hgb_3G_rotated",
    cv_mode="random", early_stopping=True)

### Step 2.2: Train DNN models

Now we repeat the exact same study for DNN models. Again, we start with the initial dataset without rotations:

In [ ]:
# Train DNN models
full_losses_dnn_0G, models_dnn_0G = train_model_multirun(
    data,
    num_runs=num_runs, ensembles_per_model=ensembles_per_model,
    max_iters=max_iters, model_type="DNN", compute_val_weights=True,
    save_model_dir="./models/models_dnn_0G",
    cv_mode="random", early_stopping=True)

And then on the dataset with the three Gaussian noise features added:

In [ ]:
# Train DNN models
full_losses_dnn_3G, models_dnn_3G = train_model_multirun(
    data_3G,
    num_runs=num_runs, ensembles_per_model=ensembles_per_model,
    max_iters=max_iters, model_type="DNN", compute_val_weights=True,
    save_model_dir="./models/models_dnn_3G",
    cv_mode="random", early_stopping=True)

Finally, we also retrain the DNN models on the rotated features. Again, we start with the initial dataset:

In [ ]:
# Train DNN models
full_losses_dnn_0G_rotated, models_dnn_0G_rotated = train_model_multirun(
    data_rotated,
    num_runs=num_runs, ensembles_per_model=ensembles_per_model,
    max_iters=max_iters, model_type="DNN", compute_val_weights=True,
    save_model_dir="./models/models_dnn_0G_rotated",
    cv_mode="random", early_stopping=True)

And then on the dataset with the three Gaussian noise features added:

In [ ]:
# Train DNN models
full_losses_dnn_3G_rotated, models_dnn_3G_rotated = train_model_multirun(
    data_3G_rotated,
    num_runs=num_runs, ensembles_per_model=ensembles_per_model,
    max_iters=max_iters, model_type="DNN", compute_val_weights=True,
    save_model_dir="./models/models_dnn_3G_rotated",
    cv_mode="random", early_stopping=True)

### Option 2: Load pre-trained models

Instead of running all the trainings, we can simply load the pre-trained models:

In [ ]:
# Extract models if not already done
if not exists("treebased_ad_files/models"):
    from utils import extract_models
    extract_models()

In [ ]:
# Important note! For DNN, the is_dnn parameter has to be set to True
models_dnn_0G = load_models_allruns("./treebased_ad_files/models/models_DNN_0G", is_dnn=True)
models_dnn_3G = load_models_allruns("./treebased_ad_files/models/models_DNN_3G", is_dnn=True)
models_dnn_0G_rotated = load_models_allruns("./treebased_ad_files/models/models_DNN_0G_rotated", is_dnn=True)
models_dnn_3G_rotated = load_models_allruns("./treebased_ad_files/models/models_DNN_3G_rotated", is_dnn=True)
models_hgb_0G = load_models_allruns("./treebased_ad_files/models/models_HGB_0G")
models_hgb_3G = load_models_allruns("./treebased_ad_files/models/models_HGB_3G")
models_hgb_0G_rotated = load_models_allruns("./treebased_ad_files/models/models_HGB_0G_rotated")
models_hgb_3G_rotated = load_models_allruns("./treebased_ad_files/models/models_HGB_3G_rotated")

### Step 2.3: Create SIC curves for all models

In [ ]:
# Define colors and linestyles for plots
colormap_gaussian = col.get_cmap("viridis")
N=6
col_3G_val = 2.5/N
col_3G = colormap_gaussian(col_3G_val)
color_list = ["black", "black", col_3G, col_3G]
linestyles = ["solid", "dashed", "solid", "dashed"]

In [ ]:
# set RC params
plt.rcParams['pgf.rcfonts'] = False
plt.rcParams['font.serif'] = []
plt.rcParams['axes.formatter.useoffset'] = False
plt.rcParams['lines.linewidth'] = 2
plt.rcParams['errorbar.capsize'] = 2
plt.rcParams['grid.linewidth'] = 0.5
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['axes.titlesize'] = 16
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
plt.rcParams['legend.title_fontsize'] = 12
plt.rcParams['legend.fontsize'] = 12
plt.rcParams['legend.frameon'] = False

In [ ]:
# Check if plots directory exists
if not exists("plots"):
    mkdir("plots")

Create SIC curves for the HGB models:

In [ ]:
plot_sic_curve_comparison([models_hgb_0G, models_hgb_0G_rotated, models_hgb_3G, models_hgb_3G_rotated],
                          [data, data_rotated, data_3G, data_3G_rotated],
                          model_types=["HGB", "HGB", "HGB", "HGB"],
                          out_filename=join("plots", "rotation_comparison_HGB.pdf"),
                          color_list=color_list,
                          linestyles=linestyles,
                          labels=["Baseline", "Baseline rotated", "Baseline + 3G", "Baseline + 3G rotated"],
                          xlabel=r"$\epsilon_{S}$",
                          ylabel=r"$\epsilon_S/\sqrt{\epsilon_B}$",
                          max_y=20,
                          title="BDT")

And for the DNN models:

In [ ]:
plot_sic_curve_comparison([models_dnn_0G, models_dnn_0G_rotated, models_dnn_3G, models_dnn_3G_rotated],
                          [data, data_rotated, data_3G, data_3G_rotated],
                          model_types=["DNN", "DNN", "DNN", "DNN"],
                          out_filename=join("plots", "rotation_comparison_DNN.pdf"),
                          color_list=color_list,
                          linestyles=linestyles,
                          labels=["Baseline", "Baseline rotated", "Baseline + 3G", "Baseline + 3G rotated"],
                          xlabel=r"$\epsilon_{S}$",
                          ylabel=r"$\epsilon_S/\sqrt{\epsilon_B}$",
                          max_y=20,
                          title="NN")